In [ ]:
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
import surp
import arya

In [ ]:
fiducial = surp.ViceModel.from_file("../../models/fiducial/run/model.json")

In [ ]:
params = surp.MWParams.from_file("../../models/fiducial/run/params.toml")

In [ ]:
h = fiducial.history

In [ ]:
t_end = np.max(h.time)

In [ ]:
h_end = h[h.time == t_end]

In [ ]:
plt.xlabel("zone")
plt.ylabel("radius / kpc")
plt.scatter(h.zone, h.R, s=1)

sanity check that radius correctly corresponds to zone number and the ranges of both make sens

# Star Formation History
Should go as the two part power law

In [ ]:
def sfh_model(t, tau_sfh, tau_rise):
    return np.exp(-t/tau_sfh) * (1 - np.exp(-t/tau_rise))

In [ ]:
tau_sfh_exp = np.vectorize(surp.simulation.star_formation_history.get_sfh_timescale)

In [ ]:
plt.scatter(h.time, h.sfr, c=h.R, s=0.01)
plt.xlabel("time")
plt.ylabel("sfr")
plt.colorbar(label="radius")

In [ ]:
tau_sfh_exp(15.05)

In [ ]:
for R, tau_sfh in [(3.05, 7.5), (7.05, 14), (15.05, 35)]:
    tau_rise = 2
    filt = np.isclose(h.R, R)
    h1 = h[filt]
    plt.plot(h1.time, h1.sfr / np.max(h1.sfr), label="model")
    y = sfh_model(h1.time, tau_sfh, tau_rise)
    plt.plot(h1.time, y/np.max(y), label="expected")
    
    plt.title(f"R = {R} kpc")
    plt.xlabel("time / Gyr")
    plt.ylabel("peak normalized SFR")
    plt.legend()
    
    plt.show()

In [ ]:
plt.scatter(h_end.R, h_end.sfr)
plt.xlabel("R / kpc")
plt.ylabel("SFR")

In [ ]:
dR = np.sort(h_end.R)[1] - np.sort(h_end.R)[0]

In [ ]:
A = (2*np.pi * h_end.R * dR).values
plt.scatter(h_end.R, h_end.mstar / A)
#plt.plot(h_end.R, h_end.mgas / A)

plt.yscale("log")
#plt.xscale("log")
plt.scatter(h_end.R, 1e9*(np.exp(-h_end.R / 2.5) + 0.27*np.exp(-h_end.R / 2.0)),s=1)
plt.ylabel(r"$\Sigma_\star$")
plt.xlabel("R")


star formation densities are okay. divergence at centre is likely a result of migration but may not even be that unphysical.
Note that the orange line is arbitrarily normalized, correct normlization is validated by total mass below...

In [ ]:
print("%0.2e" % np.sum(h_end.mstar)) # total mass should be 5.17 x 10^10 solar masses

# SFR Law

In [ ]:
def KS_law(Σg):
    # sfr density as a function of gas mass
    Σ1 = 2e7
    Σ2 = 5e6
    γ1 = 1
    γ2 = 3.6
    γ3 = 1.7
    
    C1 = Σ2**(γ3 - γ2) * Σ1**(γ2 - γ1)
    C2 = Σ2**(γ3 - γ2)
    norm = 1/2.1 / C1

    s = 0
    s += np.where(Σ1 < Σgas , C1 * Σgas ** γ1, 0)
    s += np.where((Σ2 < Σgas) & (Σgas <= Σ1), C2 * Σgas ** γ2, 0)
    s += np.where(Σgas <= Σ2, Σgas**γ3, 0)
    return norm*s

In [ ]:
Σgas = h_end.mgas/A

1/(h_end.sfr / A / Σgas * 1e9)[np.isclose(h_end.R, 7.05)]

In [ ]:
Σgas = h_end.mgas/A
yr_per_Gyr = 1e9
tau_star = h_end.mgas /(h_end.sfr * yr_per_Gyr)
plt.scatter(h_end.R, tau_star)
plt.colorbar()
Σgas = np.linspace(1e6, 1e9, 1000)
plt.xlabel(r"R / kpc")
plt.ylabel(r"$\tau_\star\,$/Gyr")
plt.title("z = 0")
plt.ylim(1)


In [ ]:
from scipy.optimize import curve_fit
tau_sfh = tau_sfh_exp(h_end.R)

def lin(x, a, b):
    return a*x + b

filt = h_end.R < 12

tau_star_0 = 2
popt, covt = curve_fit(lin, h_end.R[filt], tau_star_0 / tau_sfh[filt])

In [ ]:
tau_star[np.argmin(np.abs(h_end.R - 8.12))]

In [ ]:
min(tau_star)

In [ ]:
dR

In [ ]:

plt.scatter(h_end.R, tau_sfh)

plt.xlabel(r"R / kpc")
plt.ylabel(r"$\tau_{\rm sfh}\,/\,\rm Gyr$")

plt.scatter(h_end.R, 2/lin(h_end.R, *popt))
plt.ylim(0, 70)

In [ ]:
tau_sfh = tau_sfh_exp(h_end.R)
plt.scatter(h_end.R, np.array(tau_star) /tau_sfh)

plt.xlabel(r"R / kpc")
plt.ylabel(r"$\tau_\star / \tau_{\rm sfh}$")

plt.plot(h_end.R, lin(h_end.R, *popt))

In [ ]:
Σgas = h_end.mgas/A
plt.scatter(Σgas, 1/(h_end.sfr / A / Σgas * 1e9), c=h_end.R, zorder=5)
plt.colorbar()
Σgas = np.linspace(1e6, 1e9, 1000)
plt.scatter(Σgas, Σgas / KS_law(Σgas))
plt.xlabel(r"$\Sigma_{\rm gas}$")
plt.ylabel(r"$\tau_\star$")
plt.yscale("log")
plt.xscale('log')
plt.xlim(1e2)
plt.title("z = 0")
plt.ylim(1)


In [ ]:
np.log10(2)

In [ ]:
def log_t_dep(log_z):
    At = 0.089
    Bt = -0.62
    return  Bt*log_z # ignoring offset

def t_dep_factor(z):
    return 10**log_t_dep(np.log10(1+z))

In [ ]:
z = np.linspace(0, 3, 100)
plt.plot(z, t_dep_factor(z) * 2)
plt.xlabel("redshift")
plt.ylabel(r"$\tau_0$")

In [ ]:
filt = np.isclose(h.time, 13.2 - 7.8)
h1 = h[filt]

Σgas = h1.mgas/A
plt.scatter(Σgas, h1.sfr / A / Σgas * 1e9)
Σgas = np.linspace(1e6, 1e9, 1000)

plt.scatter(Σgas, KS_law(Σgas)/ Σgas / t_dep_factor(1))
plt.xlabel(r"$\Sigma_{\rm gas}$")
plt.ylabel(r"$\dot{\Sigma}_\star$")
plt.yscale("log")
plt.xscale('log')
plt.title("z = 1")
plt.xlim(1e5)

In [ ]:
filt = np.isclose(h.time, 13.2 - 10.4)
h1 = h[filt]

Σgas = h1.mgas/A
plt.scatter(Σgas, h1.sfr / A / Σgas * 1e9)
Σgas = np.linspace(1e6, 1e9, 1000)

plt.plot(Σgas, KS_law(Σgas)/ Σgas /  t_dep_factor(2))
plt.xlabel(r"$\Sigma_{\rm gas}$")
plt.ylabel(r"$\dot{\Sigma}_\star$")
plt.yscale("log")
plt.xscale('log')
plt.title("z = 2")
plt.xlim(1)

ks law verified !!!

In [ ]:
h.R.unique()

In [ ]:
h_solar = h[np.isclose(h.R, 8.15)]

In [ ]:
tau_star = h_solar.mgas /(h_solar.sfr * yr_per_Gyr)

plt.plot(h_solar.time, tau_star)
plt.ylim(0, 20)
plt.xlabel("time")
plt.ylabel("tau star / Gyr")
plt.axhline(2)

# Outflows and yields

In [ ]:
plt.scatter(h_end.R, h_end.eta_0)
plt.xlabel("R")
plt.ylabel(r"$\eta$")
plt.yscale("log")

In [ ]:
from surp.simulation.properties import MH_grad

In [ ]:
plt.scatter(h_end.R, h_end.M_H)
y = MH_grad(params)(h_end.R)
plt.scatter(h_end.R, y)

plt.xlim(0, 15)
plt.ylim(-0.7, 0.7)
plt.xlabel("R")
plt.ylabel(r"[M/H]")

In [ ]:
filt = h.R < 5
df = h[filt]
plt.scatter(df.time, df.MG_H, s=0.3, alpha=0.1, c=df.R)
plt.xlim(0, 15)
plt.ylim(-0.5, 0.5)
plt.colorbar()

# Migration

In [ ]:
s = fiducial.stars_unsampled

In [ ]:
r_bins = [1, 3, 5, 7, 9, 13]
Nbins = len(r_bins) - 1
for i in range(Nbins):
    filt = s.r_origin >= r_bins[i]
    filt &= s.r_origin < r_bins[i+1]
    plt.hist(s[filt].r_final, 30, histtype="step", density=True, label=f"$R_i \in {r_bins[i]}, {r_bins[i+1]}$")

plt.xlabel(r"$R_f$ / kpc")
plt.ylabel("density")

arya.Legend(-1)

In [ ]:
r_bins = [0, 0.1, 0.6, 1]
Nbins = len(r_bins) - 1
for i in range(Nbins):
    filt = s.r_origin >= r_bins[i]
    filt &= s.r_origin < r_bins[i+1]
    plt.hist(s[filt].r_final, 30, histtype="step", density=True, label=f"$R_i \in {r_bins[i]}, {r_bins[i+1]}$")

plt.xlabel(r"$R_f$ / kpc")
plt.ylabel("density")

arya.Legend(-1)

In [ ]:
plt.hist(s.r_final, 200, ec="none");